In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial
import os

# Создаём папку "NRMSE", если её нет
if not os.path.exists("NRMSE"):
    os.makedirs("NRMSE")

# Вспомогательная функция для вычисления радиальной части полиномов Цернике
def R(n, m, r):
    """Функция для расчета радиальной части полинома Цернике"""
    if (n - m) % 2 != 0:
        return np.zeros_like(r)
    radial_sum = np.zeros_like(r)
    for k in range((n - m) // 2 + 1):
        c = (-1) ** k * factorial(n - k) / (factorial(k) * factorial((n + m) // 2 - k) * factorial((n - m) // 2 - k))
        radial_sum += c * r ** (n - 2 * k)
    return radial_sum

# Функция для обрезки изображения до центральной части 128x128
def crop_center(img, crop_size=128):
    h, w = img.shape
    startx = w // 2 - (crop_size // 2)
    starty = h // 2 - (crop_size // 2)    
    return img[starty:starty + crop_size, startx:startx + crop_size]

# Основная функция для вычисления полиномов Цернике
def zernike_polynomial(n, m, rho, theta):
    """Вычисление полинома Цернике"""
    if m >= 0:
        return R(n, m, rho) * np.cos(m * theta)
    else:
        return R(n, -m, rho) * np.sin(-m * theta)

# Функция для вычисления нормированного СКО в процентах
def calculate_nrmse(img1, img2):
    """Вычисление NRMSE в процентах относительно максимума img1 (Zf)"""
    diff = np.abs(img1) - np.abs(img2)
    rmse = np.sqrt(np.mean(diff ** 2))
    max_img1 = np.max(np.abs(img1))
    nrmse_percent = (rmse / max_img1) * 100 if max_img1 != 0 else 0
    return nrmse_percent

# Основная функция для генерации и сохранения
def generate_zernike_images(n_m_pairs, a_values, b=20, grid_size=100, fft_size=512):
    """Генерация изображений и вычисление NRMSE для всех комбинаций"""
    for n, m in n_m_pairs:
        for a in a_values:
            # Генерация координат
            y, x = np.linspace(-1, 1, grid_size), np.linspace(-1, 1, grid_size)
            X, Y = np.meshgrid(x, y)
            rho = np.sqrt(X**2 + Y**2)
            theta = np.arctan2(Y, X)
            
            # Ограничиваем область единичным кругом
            mask = rho <= 1
            rho[~mask] = 0
            theta[~mask] = 0
            
            # Вычисление полинома и фаз
            Z = zernike_polynomial(n, m, rho, theta)
            Z1a = np.cos((a * Z + b * np.sqrt(X ** 2 + Y ** 2)))  # С аксиконом

            # Фурье-преобразования
            Zf = np.fft.fftshift(np.fft.fft2(Z, s=[fft_size, fft_size]))  # Идеальная картина
            Z1af = np.fft.fftshift(np.fft.fft2(Z1a, s=[fft_size, fft_size]))  # С аксиконом

            # Обрезаем до 128x128
            Zf_cropped = crop_center(abs(Zf))
            Z1af_cropped = crop_center(abs(Z1af))

            # Вычисляем NRMSE
            nrmse_percent = calculate_nrmse(Zf_cropped, Z1af_cropped)

            # Выводим NRMSE в консоль
            print(f"n={n}, m={m}, a={a}, b={b}: NRMSE = {nrmse_percent:.2f}%")

            # Сохраняем изображение ФРТ
            filename = f"NRMSE/zernike_n{n}_m{m}_a{a}_b{b}_FRT.png"
            plt.imsave(filename, Z1af_cropped, cmap='gray')

# Список пар (n, m) и значений a
n_m_pairs = [
    (1, 1),  # Смещение
    (2, 2),  # Астигматизм
    (2, 0),  # Сферическая
    (3, 3),  # Трилистник
    (3, 1),  # Кома
    (4, 4),  # Четырёхлистник
    (4, 2),  # Астигматизм 2-го порядка
    (4, 0),  # Сферическая
]

a_values = [0, 1, 5, 10, 25, 50]


In [2]:
generate_zernike_images(n_m_pairs, a_values, b=20)

n=1, m=1, a=0, b=20: NRMSE = 10.75%
n=1, m=1, a=1, b=20: NRMSE = 10.48%
n=1, m=1, a=5, b=20: NRMSE = 9.72%
n=1, m=1, a=10, b=20: NRMSE = 9.63%
n=1, m=1, a=25, b=20: NRMSE = 8.43%
n=1, m=1, a=50, b=20: NRMSE = 7.57%
n=2, m=2, a=0, b=20: NRMSE = 16.47%
n=2, m=2, a=1, b=20: NRMSE = 16.22%
n=2, m=2, a=5, b=20: NRMSE = 15.50%
n=2, m=2, a=10, b=20: NRMSE = 14.88%
n=2, m=2, a=25, b=20: NRMSE = 12.82%
n=2, m=2, a=50, b=20: NRMSE = 10.33%
n=2, m=0, a=0, b=20: NRMSE = 12.13%
n=2, m=0, a=1, b=20: NRMSE = 11.06%
n=2, m=0, a=5, b=20: NRMSE = 11.49%
n=2, m=0, a=10, b=20: NRMSE = 10.49%
n=2, m=0, a=25, b=20: NRMSE = 9.33%
n=2, m=0, a=50, b=20: NRMSE = 9.18%
n=3, m=3, a=0, b=20: NRMSE = 22.32%
n=3, m=3, a=1, b=20: NRMSE = 21.74%
n=3, m=3, a=5, b=20: NRMSE = 19.97%
n=3, m=3, a=10, b=20: NRMSE = 19.44%
n=3, m=3, a=25, b=20: NRMSE = 15.79%
n=3, m=3, a=50, b=20: NRMSE = 13.34%
n=3, m=1, a=0, b=20: NRMSE = 22.41%
n=3, m=1, a=1, b=20: NRMSE = 21.88%
n=3, m=1, a=5, b=20: NRMSE = 18.73%
n=3, m=1, a=10, b=20: 